<a href="https://colab.research.google.com/github/dr-kinder/playground/blob/dev/10_diffusion_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NOTE #
It appears that the `multiphenicsx.io` library and the `pyvista.start_xvfb()` commands are incompatible.  I suspect the problem is the `itkwidgets` library used in multiphenicsx.

Whatever the cause, the effect is that I cannot generate movies with PyVista if I import **multiphenicsx**; I can if I don't.  So don't.

# Introduction

This notebook introduce the time dependent equations of electrodynamics.

We will use Maxwell's equations to explore how the charge, potentials, and fields relax to equilibrium.  Refer to Sections 4.1, 4.4, and 4.5 of our textbook for background on the equations for the fields.

Refer to the [`colab_movies.ipynb`](https://github.com/dr-kinder/playground/blob/dev/colab_movies.ipynb) notebook for more explanation of some details of the code.

# Install and Import

In [ ]:
from IPython import display

In [ ]:
# Need to upgrade matplotlib for some PyVista plotting commands to work.
# Run this cell, then select "Runtime > Restart runtime" from the CoLab menu.
!pip install --upgrade matplotlib

display.clear_output()

In [ ]:
try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

display.clear_output()

In [ ]:
# This simulation uses real values.
try:
    import dolfinx
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

display.clear_output()

In [ ]:
# Everything should be installed now.
# Import the rest of what we need.
import gmsh

from dolfinx import fem, mesh, plot, la
from dolfinx.io import gmshio
import ufl

from mpi4py import MPI
import petsc4py.PETSc as petsc

import numpy as np
import pyvista

# Start a virtual plot window for PyVista in CoLab.
pyvista.start_xvfb()
pyvista.set_jupyter_backend("pythreejs")

# Get tools for embedding movies in CoLab.
from base64 import b64encode

# Initial Conditions

The code in this section is similar to [06-finite-element-method.ipynb](https://github.com/dr-kinder/electrodynamics/blob/master/week-03/06-finite-element-method.ipynb).  We will use the solution to the electrostatics problem as the initial condition for an electrodynamics problem.

## Model

The model is a gaussian charge distribution inside of a box with grounded walls.

You can change the boundary conditions to explore other systems.

In [ ]:
# Create a simple rectangular mesh.
length = 10
height = 10
Nx, Ny = 51, 51
extent = [[-length/2, -height/2], [length/2, height/2]]
domain = mesh.create_rectangle(
    MPI.COMM_WORLD, extent, [Nx, Ny], mesh.CellType.triangle)

In [ ]:
# Prepare the mesh for plotting.
topology, cells, geometry = dolfinx.plot.create_vtk_mesh(domain)

# Turn the mesh into a PyVista grid.
grid = pyvista.UnstructuredGrid(topology, cells, geometry)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(grid, show_edges=True)
plotter.view_xy()

# Save the HTML file.
plotter.export_html("./grid.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/grid.html')

## Charge Density

This time, we will define the charge density as a time-dependent function on the grid.

In [ ]:
# Define a set of functions on our mesh.
V = dolfinx.fem.FunctionSpace(domain, ("Lagrange", 2))

In [ ]:
# Define the charge.
xC = -1
yC = -1
Q = 1
sQ = 0.2
beta = 1/sQ
tau = 1

# This function returns a function that can be interpolated.
def rho_function(t):
    return lambda x: Q * beta / 2 / np.pi * np.exp(-t/tau) * np.exp(-0.5 * beta**2 * ((x[0]-xC)**2 + (x[1] - yC)**2))

# Turn this function definition into a time-dependent function on the mesh.
rho = fem.Function(V)

# Initialize the function for t=0.
rho.interpolate(rho_function(0))

In [ ]:
# Plot the initial charge density.
topology, cells, geometry = plot.create_vtk_mesh(rho.function_space)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["rho"] = rho.x.array
grid.set_active_scalars("rho")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
warped = grid.warp_by_scalar("rho", factor=10)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Charge Density"},
    clim=[0, 1],
    cmap='turbo'
)
plotter.add_text("Initial Charge Density")
# Save the HTML file.
plotter.export_html("./rho-initial.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/rho-initial.html')

## Boundary Conditions

We will set the potential to zero on the boundary to compute the initial potential and fields.  After this, the boundaries will be open.

In [ ]:
# Define a function to locate boundaries.
# It will return True for points on the boundary, and False otherwise.
def find_facets(x):
    yes = np.isclose(x[0], -length/2)
    yes += np.isclose(x[0], length/2)
    yes += np.isclose(x[1], -height/2)
    yes += np.isclose(x[1], height/2)
    return yes

# Next, we use this function to tag each cell along the boundary.
tdim = domain.topology.dim
bc_facets = mesh.locate_entities_boundary(
    domain, tdim - 1, find_facets)

# Identify those for which the function was True as
# boundary_dofs = "boundary degrees of freedom"
boundary_dofs = fem.locate_dofs_topological(V, tdim - 1, bc_facets)

# # Now introduce the boundary condition: constant potential on the boundary.
# Define the potential on the boundary.
def uD_function(t):
    return lambda x: 0 * x[0]

# Turn this function definition into a time-dependent function on the mesh.
uD = fem.Function(V)

# Initialize the function for t=0.
uD.interpolate(uD_function(0.0))

# Add these to the list of boundary conditions to be imposed.
# This will apply the uD function to the cells along the boundary.
# It will be updated during each time step.
bcs = [fem.dirichletbc(uD, boundary_dofs)]

## Solve for the Initial Potential

In [ ]:
# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
phi = fem.Function(V)
phi_n = fem.Function(V)

# This is the FEM version of the Laplacian.
# It is the left-hand side of Poisson's equation.
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

# This is how we introduce the charge density.
# It is the right-hand side of Poisson's equation.
L = 4 * ufl.pi * rho * v * ufl.dx

# Put it all together for FEniCSx.
problem = fem.petsc.LinearProblem(a, L, bcs=bcs, u=phi, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})

# Now, solve it!
problem.solve()

# Tie up some loose ends.
phi.vector.ghostUpdate(addv=petsc.InsertMode.INSERT, mode=petsc.ScatterMode.FORWARD)

In [ ]:
# Plot the initial potential.
topology, cells, geometry = plot.create_vtk_mesh(rho.function_space)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["phi"] = phi.x.array
grid.set_active_scalars("phi")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
warped = grid.warp_by_scalar("phi", factor=10)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Potential"},
    clim=[0, 1],
    cmap='turbo'
)
plotter.add_text("Initial Potential")
# Save the HTML file.
plotter.export_html("./phi-initial.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/phi-initial.html')

In [ ]:
# Define a set of elements for a vector field.
W = dolfinx.fem.VectorFunctionSpace(domain, ("Lagrange", 2))
E = dolfinx.fem.Function(W)

# Compute the gradient as a symbolic expression, then interpolate it onto the mesh.
expr = dolfinx.fem.Expression(ufl.as_vector((-phi.dx(0), -phi.dx(1))), W.element.interpolation_points())
E.interpolate(expr)

In [ ]:
# Plot the initial electric field.
topology, cells, geometry = plot.create_vtk_mesh(E.function_space)
values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
values[:, :len(E)] = E.x.array.real.reshape((geometry.shape[0], len(E)))

# Create a point cloud of glyphs
e_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
e_grid["E"] = values
arrows = e_grid.glyph(orient="E", factor=0.2)

# Create a pyvista-grid for the mesh
topology, cells, geometry = plot.create_vtk_mesh(domain, domain.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)

# Create plotter
plotter = pyvista.Plotter()
plotter.add_mesh(grid, style="wireframe", color='#BBBBBB')
plotter.background_color = 'white'
plotter.add_mesh(arrows)
plotter.add_text("Initial Field")
# Save the HTML file.
plotter.export_html("./e-initial.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/e-initial.html')

In [ ]:
# Store this solution for reuse later.
phi_initial = phi.copy()

## Questions

In Notebook #6, we looked at a similar charge distribution in a circular geometry.  This time we placed the charge distribution in a rectangular geometry.

***Describe the similarities and differences between the resulting potential.***

REPLACE WITH YOUR RESPONSE.

# Slow Relaxation

We now let the system relax to equilibrium.  If the system is weakly conducting, then relaxation to equilibrium is slow enough that conduction and displacement currents cancel each other and there is no magnetic field.  (See Heald & Marion 4.4.)

In this case, the problem is "quasi-static".  We can solve the time *dependent* problem (approximately) by solving a time *independent* problem at each time step, using the current value of $rho(t)$.

The continuity equation and Ohm's Law imply that
$$\rho(t) = \rho_0 \, e^{-t/\tau}$$
where the relaxation time $\tau$ is
$$\tau = \dfrac{4\pi\sigma}{\epsilon}$$

We can visualize this by placing the FEM solving code inside of a loop.

## Setup

You can adjust the parameters below to control the time step and duration of the simulation.

In [ ]:
# This is the time step for the simulation.
dt = fem.Constant(domain, 0.01)

# Initial time and maximum time.
t0 = 0.0
t_max = 5*tau

In [ ]:
# Constant factor from Maxwell's equations.
k = fem.Constant(domain, 4*np.pi)

# This is the problem we are going to solve:
# We update the charge distribution and compute the potential
# using Poisson's equation at each time step.
F = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
F -= k * ufl.inner(rho, v) * ufl.dx
(a, L) = ufl.system(F)

In [ ]:
# Building the matrix for the finite element problem takes a while.
# It does not change throughout the problem, either.  The following
# line build the matrix once and store it for reuse.
compiled_a = fem.form(a)
A = fem.petsc.assemble_matrix(compiled_a, bcs=bcs)
A.assemble()

In [ ]:
# We do the same for the vector, but it will be updated at each time step.
compiled_L = fem.form(L)
b = fem.Function(V)

In [ ]:
# Now we use the PETSc problem to construct the linear algebra problem.
solver = petsc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(petsc.KSP.Type.CG)
pc = solver.getPC()
pc.setType(petsc.PC.Type.HYPRE)
pc.setHYPREType("boomeramg")

## Simulation

The next cells will make movies of the charge distribution and the potential as the charge distribution "slowly" relaxes to equilibirium: zero net charge inside the conductor.

In [ ]:
# Prepare the mesh for PyVista ...
topology, cells, geometry = plot.create_vtk_mesh(V)

# Create a plotter for charge density, but don't display the plot.
rho_file = "%03d-charge.jpg"

# It will be making a movie in the background.
rho_plotter = pyvista.Plotter(notebook=False, off_screen=True)

# And add the charge and potentials to the plot.
topology, cells, geometry = plot.create_vtk_mesh(rho.function_space)
rho_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
rho_grid.point_data["rho"] = rho.x.array
rho_grid.set_active_scalars("rho")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
rho_warped = rho_grid.warp_by_scalar("rho", factor=10)

# Add the warped grid to the plot.
rho_plotter.add_mesh(
    rho_warped,
    lighting=False,
    show_edges=False,
    clim=[0, 1],
    cmap='turbo'
);

In [ ]:
# Do exactly the same thing for the potential.
phi_file = "%03d-potential.jpg"
phi_plotter = pyvista.Plotter(off_screen=True, notebook=False)
phi_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
phi_grid.point_data["phi"] = phi.x.array
phi_warped = phi_grid.warp_by_scalar("phi", factor=10)
phi_plotter.add_mesh(
    phi_warped,
    lighting=False,
    show_edges=False,
    clim=[0, 1],
    cmap='turbo'
);

In [ ]:
# Get rid of any old movie files.
!rm *-charge.jpg
!rm *-potential.jpg
!rm charge-movie.mp4
!rm potential-movie.mp4

In [ ]:
# Now, compute an update for each time step and add it to the movie.
# Initial time
t = t0
n = 0

# Set up the initial potential.
phi = phi_initial.copy()

# Loop until completion.
while t < t_max:
    # Take snapshots for the video.
    rho_frame = rho_file % n
    rho_plotter.show(screenshot=rho_frame, auto_close=False)
    phi_frame = phi_file % n
    phi_plotter.show(screenshot=phi_frame, auto_close=False) 
    
    # Update boundary conditions.
    t += dt.value
    rho.interpolate(rho_function(t))
    uD.interpolate(uD_function(t))

    # Assemble the RSH: the vector on the "right-hand side" of A.u = b.
    b.x.array[:] = 0
    fem.petsc.assemble_vector(b.vector, compiled_L)

    # Apply boundary condition.
    # These commands distribute the updated problem to all processes.
    fem.petsc.apply_lifting(b.vector, [compiled_a], [bcs])
    b.x.scatter_reverse(la.ScatterMode.add)
    fem.petsc.set_bc(b.vector, bcs)

    # Solve linear problem.
    solver.solve(b.vector, phi.vector)

    # Distribute the solution to all processes.
    phi.x.scatter_forward()

    # Update un --- the current value of the function.
    phi_n.x.array[:] = phi.x.array

    # Update the plots and save the frame to the GIF.
    rho_grid.point_data["rho"] = rho.x.array
    rho_warped = rho_grid.warp_by_scalar("rho", factor=10)
    rho_plotter.update_scalars(rho.x.array, render=False)
    rho_plotter.update_coordinates(rho_warped.points)

    phi_grid.point_data["phi"] = phi.x.array
    phi_warped = phi_grid.warp_by_scalar("phi", factor=10)
    phi_plotter.update_scalars(phi.x.array, render=False)
    phi_plotter.update_coordinates(phi_warped.points, render=False)
    phi_frame = phi_file % n

    n += 1

# Take final snapshots for the video.
rho_frame = rho_file % n
rho_plotter.show(screenshot=rho_frame, auto_close=False)
phi_frame = phi_file % n
phi_plotter.show(screenshot=phi_frame, auto_close=False)

# Close the plotter and finish processing the movie.
rho_plotter.close()
phi_plotter.close()

In [ ]:
# Assemble the movies with FFMPEG.
!ffmpeg -y -i %03d-charge.jpg -pix_fmt yuv420p charge-movie.mp4
!ffmpeg -y -i %03d-potential.jpg -pix_fmt yuv420p potential-movie.mp4

## Results

The two movies below show the charge distribution and the potential evolving in time.

In [ ]:
# Play the charge movie.
mp4 = open('charge-movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# Play the potential movie.
mp4 = open('potential-movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Questions

- Describe what happens as the charge distribution relaxes to equilibrium.
- Describe any correlations or significant differences you see between the behavior of the charge distribution and the potential.
- Describe any interesting or unusual features you observe in the simulations.

***Summarize your observations below.***

REPLACE WITH YOUR RESPONSE.

# Field Diffusion

In a good conductor, the charges and the electromagnetic fields do not remain in synch.  Rapidly changing electromagnetic fields lead to induction, and the field may take much longer to reach equilibrium than the charges.

Heald an Marion show that the field equations become diffusion equations:
\begin{align*}
\nabla^2 \vec{E} - \dfrac{4\pi\mu\sigma}{c^2} \dfrac{\partial \vec{E}}{\partial t} &= \dfrac{4\pi}{\epsilon} \nabla \rho \\
\nabla^2 \vec{B} - \dfrac{4\pi\mu\sigma}{c^2} \dfrac{\partial \vec{B}}{\partial t} &= 0
\end{align*}

The ***diffusion equation*** is
$$D \nabla^2 \phi - \dfrac{\partial \phi}{\partial t} = 0$$
Thus, the fields obey a diffusion equation with diffusion constant
$$
D = \dfrac{c^2}{4\pi\mu\sigma}
$$

As Marion and Heald point out, the timescale for diffusion over a distance of 1 mm in copper is on the order of 0.1 ms, compared with $10^{-14}$ s for the relaxation of the charge to its equilibrium distribution.  The relaxation of the fields in copper will take about 10 billion times longer than the relaxation of the charge distribution we just simulated.  Thus, we will ignore the charge density in what follows.

The following cells return to our initial potential, but allow it to relax under the action of the diffusion equation, rather than tracking the density.  The components of the electromagnetic field will behave in a similar fashion.

The cells below are a copy of those in the preceding section, except the finite element problem definition is now that appropriate to a diffusion equation.

## Setup

In [ ]:
# Diffusion constant.
d_val = 10.0

# Initial time and maximum time.
t0 = 0.0
t_max = 1
t_step = 0.002

# This is the time step for the simulation.
D = fem.Constant(domain, d_val)
dt = fem.Constant(domain, t_step)
f = fem.Constant(domain, 0.0)

In [ ]:
# Reset the initial potential.
phi = phi_initial.copy()
phi_n = fem.Function(V)

In [ ]:
# This is the problem we are going to solve:
# The diffusion equaiton, in weak form for finite elements.
F = ufl.inner(u - phi_n, v) * ufl.dx
F += dt * D * ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
F -= dt * ufl.inner(f, v) * ufl.dx
(a, L) = ufl.system(F)

In [ ]:
## Choose boundary conditions.
bcs = [fem.dirichletbc(uD, boundary_dofs)]
# bcs = []

In [ ]:
# Building the matrix for the finite element problem takes a while.
# It does not change throughout the problem, either.  The following
# line build the matrix once and store it for reuse.
compiled_a = fem.form(a)
A = fem.petsc.assemble_matrix(compiled_a, bcs=bcs)
A.assemble()

In [ ]:
# We do the same for the vector, but it will be updated at each time step.
compiled_L = fem.form(L)
b = fem.Function(V)

In [ ]:
# Now we use the PETSc problem to construct the linear algebra problem.
solver = petsc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(petsc.KSP.Type.CG)
pc = solver.getPC()
pc.setType(petsc.PC.Type.HYPRE)
pc.setHYPREType("boomeramg")

## Simulation

In [ ]:
# Create a plotter for charge density, but don't display the plot.
phi_file = "%03d-diffusion.jpg"

# It will be making a movie in the background.
phi_plotter = pyvista.Plotter(notebook=False, off_screen=True)

# And add the charge and potentials to the plot.
topology, cells, geometry = plot.create_vtk_mesh(phi.function_space)
phi_grid = pyvista.UnstructuredGrid(topology, cells, geometry)
phi_grid.point_data["phi"] = phi.x.array
phi_grid.set_active_scalars("phi")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
phi_warped = phi_grid.warp_by_scalar("phi", factor=10)

# Add the warped grid to the plot.
phi_plotter.add_mesh(
    phi_warped,
    lighting=False,
    show_edges=False,
    clim=[0, 1],
    cmap='turbo'
);

In [ ]:
# Plot the initial potential.
topology, cells, geometry = plot.create_vtk_mesh(phi.function_space)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["phi"] = phi.x.array
grid.set_active_scalars("phi")

# The grid is flat.  This will warp the grid into three dimensions
# using the value of the function as the height.
warped = grid.warp_by_scalar("phi", factor=10)

# Create the plot and export it to HTML.
plotter = pyvista.Plotter(window_size=(800, 400))
renderer = plotter.add_mesh(
    warped,
    lighting=False,
    show_edges=False,
    scalar_bar_args={"title": "Potential"},
    clim=[0, 1],
    cmap='turbo'
)
plotter.add_text("Initial Potential")
# Save the HTML file.
plotter.export_html("./phi-initial.html", backend="pythreejs")

# Use the IPython library to embed the HTML in the CoLab notebook.
display.HTML(filename='/content/phi-initial.html')

In [ ]:
# Get rid of old diffusion movie files.
!rm *-diffusion.jpg
!rm diffusion-movie.mp4

In [ ]:
# Now, compute an update for each time step and add it to the movie.
# Initial time
t = t0
n = 0

# Set up the initial potential.
phi = phi_initial.copy()
phi_n.x.array[:] = phi_initial.x.array

# Loop until completion.
while t < t_max:
    # Take snapshots for the video.
    phi_frame = phi_file % n
    phi_plotter.show(screenshot=phi_frame, auto_close=False) 
    
    # Update boundary conditions.
    t += dt.value
    rho.interpolate(rho_function(t))
    uD.interpolate(uD_function(t))

    # Assemble the RSH: the vector on the "right-hand side" of A.u = b.
    b.x.array[:] = 0
    fem.petsc.assemble_vector(b.vector, compiled_L)

    # Apply boundary condition.
    # These commands distribute the updated problem to all processes.
    fem.petsc.apply_lifting(b.vector, [compiled_a], [bcs])
    b.x.scatter_reverse(la.ScatterMode.add)
    fem.petsc.set_bc(b.vector, bcs)

    # Solve linear problem.
    solver.solve(b.vector, phi.vector)

    # Distribute the solution to all processes.
    phi.x.scatter_forward()

    # Update un --- the current value of the function.
    phi_n.x.array[:] = phi.x.array

    # Update the plots and save the frame.
    phi_grid.point_data["phi"] = phi.x.array
    phi_warped = phi_grid.warp_by_scalar("phi", factor=10)
    phi_plotter.update_scalars(phi.x.array, render=False)
    phi_plotter.update_coordinates(phi_warped.points, render=False)

    n += 1

# Take final snapshots for the video.
phi_frame = phi_file % n
phi_plotter.show(screenshot=phi_frame, auto_close=False)

# Close the plotter and finish processing the movie.
phi_plotter.close()

In [ ]:
!ffmpeg -y -i %03d-diffusion.jpg -pix_fmt yuv420p diffusion-movie.mp4

## Results

In [ ]:
# Play the diffusion movie.
mp4 = open('diffusion-movie.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Questions

- Describe similarities and differences between the diffusion simulation in this section and the "slow relaxation" simulation in the previous section.

(Keep in mind that the diffusion process actually takes over a billion times longer than the relaxation process in the previous section.)

***Describe your observations below.***

REPLACE WITH YOUR RESPONSE.

### Boundary Conditions

Find the cell above that begins

```## Choose boundary conditions```

Comment out one set of boundary condtions and uncomment the other.  This will switch the boundary condtions from essential ($\phi = 0$ on the boundaries) to natural ($\hat{n} \cdot \nabla \phi = 0$ on the boundaries).

Run all the cells in this section — "Field Diffusion" — again.

Describe similarities and differences of diffusion with closed and open boundaries.

***Describe your observations below.***

REPLACE WITH YOUR RESPONSE.

### Charge Location

Go back to the "Charge Density" section above.  Move the center of the charge closer to an edge or corner of the box by adjusting `xC` and `yC`.  Run the entire notebook again.

What effects of the boundaries do you notice when the charge is close to the wall?

***Describe your observations below.***

REPLACE WITH YOUR RESPONSE.

# Exploration

Do something original with this notebook.  You can try one the following if you are not sure where to start:

- Change the shape of the box.
- Add more charges — positive or negative — to the box.
- Explore the effect of changing the diffusion constant.
- Try to make a movie of the electric field for the diffusing potential.  (Challenging!)

***Describe your efforts and observations below.***

REPLACE WITH YOUR RESPONSE

# Reflection and Summary

- What are the major takeaways of this assignment for you?
- What was the most difficult part of this assignment?
- What was the most interesting part of this assignment?
- What questions do you have?

REPLACE WITH YOUR RESPONSE